# Gemini Data Extraction from In-Store Images

In [1]:
! pip install deepdiff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 3.7 MB/s eta 0:00:00


### Imports and Setup

This section handles the initial setup of the environment. It imports the necessary libraries for interacting with the Gemini API and Google Cloud services. It also initializes the Gemini client with the project and location details, which is a prerequisite for making any API calls.


In [19]:
import pandas as pd
import os
import vertexai
from vertexai.evaluation import EvalTask, PointwiseMetric, PointwiseMetricPromptTemplate
from google.cloud import aiplatform

PROJECT_ID = "sandbox-401718"  # @param {type:"string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")
EXPERIMENT_NAME = "fuzzy_match_eval" # @param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
)

## Accuracy Table Calculations

Compars Gemini outputs to Ground Truth labels. Output tables from 01-compscan-gemini-exp.ipynb and [TBD GROUND TRUTH CSV FILE] are used

### Import Predictions & Ground truth data

In [4]:
import json

In [9]:
file_path_gt = '../instore_image_GT.json'  # @param
file_path_pred = '../instore_image_predict.json'  # @param

with open(file_path_gt, 'r') as file:
    gt_dict = json.load(file)
    
with open(file_path_pred, 'r') as file:
    pred_dict = json.load(file)

### Product Values Accuracy

Meausres Exact Match. Exampe: Price, Size, Units, Brand

JSON comparison of exact matches using DeepDiff

In [ ]:
import pandas as pd
from deepdiff import DeepDiff
from deepdiff.path import extract

In [39]:
# LONG FORMAT

import pandas as pd
from deepdiff import DeepDiff
from deepdiff.path import extract

# Assume gt_dict and pred_dict are already defined.

# Define fields and initialize results list ---
fields_to_compare = {"Price", "Size", "UnitOfMeasure", "Brand"}
comparison_results = []

# Iterate through each file in the ground truth ---
for filename, gt_data in gt_dict.items():
    # Handle case where the prediction file is missing entirely
    if filename not in pred_dict:
        comparison_results.append({
            "Filename": filename,
            "Status": "FAIL (File Missing)",
            "Field": "N/A", "Ground Truth": "N/A", "Prediction": "N/A"
        })
        continue

    pred_data = pred_dict[filename]
    
    # Generate the difference object once per file
    diff = DeepDiff(
        gt_data,
        pred_data,
        ignore_order=True,
        ignore_string_case=True,
        ignore_numeric_type_changes=True
    )

    # Iterate through GT products to check for PASS or FAIL status ---
    if 'products' in gt_data:
        for i, gt_product in enumerate(gt_data['products']):
            
            # First, check if this entire product was reported as removed
            was_product_removed = any(
                gt_product == item for item in diff.get('iterable_item_removed', {}).values()
            )

            if was_product_removed:
                # If the whole product is missing, mark all its relevant fields as failed
                for field in fields_to_compare:
                    if field in gt_product:
                        comparison_results.append({
                            "Filename": filename,
                            "Status": "FAIL (Product Missing)",
                            "Field": field,
                            "Ground Truth": gt_product.get(field),
                            "Prediction": "N/A"
                        })
                continue # Move to the next product

            # If the product was not removed, check each field
            for field in fields_to_compare:
                if field in gt_product:
                    path_str = f"root['products'][{i}]['{field}']"
                    gt_value = gt_product[field]
                    
                    # Check for a value mismatch
                    if path_str in diff.get('values_changed', {}):
                        changes = diff['values_changed'][path_str]
                        comparison_results.append({
                            "Filename": filename,
                            "Status": "FAIL (Value Mismatch)",
                            "Field": field,
                            "Ground Truth": changes.get('old_value'),
                            "Prediction": changes.get('new_value')
                        })
                    # Check if the field was removed from the product
                    elif path_str in diff.get('dictionary_item_removed', []):
                         comparison_results.append({
                            "Filename": filename,
                            "Status": "FAIL (Field Missing)",
                            "Field": field,
                            "Ground Truth": gt_value,
                            "Prediction": "N/A"
                        })
                    else:
                        # If no diff was found for this path, it's a PASS
                        comparison_results.append({
                            "Filename": filename,
                            "Status": "PASS",
                            "Field": field,
                            "Ground Truth": gt_value,
                            "Prediction": gt_value # Value is the same
                        })
    
    # Report items that were added in the prediction ---
    if 'iterable_item_added' in diff:
        for path, item in diff['iterable_item_added'].items():
            comparison_results.append({
                "Filename": filename,
                "Status": "FAIL (Product Added)",
                "Field": "entire product",
                "Ground Truth": "N/A",
                "Prediction": item
            })
    
    if 'dictionary_item_added' in diff:
        for path in diff['dictionary_item_added']:
            field_name = path.split("'")[-2]
            if field_name not in fields_to_compare: # Often for fields like 'UPC'
                comparison_results.append({
                    "Filename": filename,
                    "Status": "INFO (Field Added)",
                    "Field": field_name,
                    "Ground Truth": "N/A",
                    "Prediction": extract(pred_data, path)
                })

# --- 5. Create and display the final DataFrame ---
df = pd.DataFrame(comparison_results)

# Reorder columns for better readability
if not df.empty:
    df = df[["Filename", "Status", "Field", "Ground Truth", "Prediction"]]

# Sort the results to group by filename and status
df_long = df.sort_values(by=["Filename", "Status"], ascending=[True, False]).reset_index(drop=True)

In [41]:
# WIDE FORMAT

wide_format_rows = []

# Iterate through each file in the ground truth
for filename, gt_data in gt_dict.items():
    row_data = {
        "Filename": filename,
        "Price": "PASS",
        "Size": "PASS",
        "UnitOfMeasure": "PASS",
        "Brand": "PASS",
        "Description": [],
        "Other_Changes": []
    }

    # Handle the case where the prediction file is missing entirely ---
    if filename not in pred_dict:
        row_data["Price"] = "FAIL (File Missing)"
        row_data["Size"] = "FAIL (File Missing)"
        row_data["UnitOfMeasure"] = "FAIL (File Missing)"
        row_data["Brand"] = "FAIL (File Missing)"
        wide_format_rows.append(row_data)
        continue  # Move to the next file

    pred_data = pred_dict[filename]
    
    # Generate the difference object once per file
    diff = DeepDiff(
        gt_data,
        pred_data,
        ignore_order=True,
        ignore_string_case=True,
        ignore_numeric_type_changes=True
    )

    # Aggregate Description fields for this file ---
    gt_products = gt_data.get('products', [])
    pred_products = pred_data.get('products', [])
    max_len = max(len(gt_products), len(pred_products))
    for i in range(max_len):
        gt_desc = gt_products[i].get('Description', 'N/A') if i < len(gt_products) else "N/A"
        pred_desc = pred_products[i].get('Description', 'N/A') if i < len(pred_products) else "N/A"
        row_data["Description"].append({"ground_truth": gt_desc, "prediction": pred_desc})

    # Determine the final PASS/FAIL status for each field
    if diff:  # Only check for failures if DeepDiff found any changes
        fields_to_check = {"Price", "Size", "UnitOfMeasure", "Brand"}
        
        # A missing or extra product causes an automatic failure for all fields.
        if 'iterable_item_added' in diff or 'iterable_item_removed' in diff:
            for field in fields_to_check:
                row_data[field] = "FAIL"
            if 'iterable_item_added' in diff:
                row_data["Other_Changes"].append(f"{len(diff['iterable_item_added'])} product(s) added")
            if 'iterable_item_removed' in diff:
                row_data["Other_Changes"].append(f"{len(diff['iterable_item_removed'])} product(s) removed")
        
        # Check for specific value changes or missing/added keys
        for change_type, paths in diff.items():
            if change_type not in ['values_changed', 'dictionary_item_removed', 'dictionary_item_added']:
                continue
            for path in paths:
                for field in fields_to_check:
                    if f"'{field}'" in path:
                        row_data[field] = "FAIL"
        
        # Capture informational changes like a UPC being added
        for path in diff.get('dictionary_item_added', []):
            field_name = path.split("'")[-2]
            if field_name not in fields_to_check:
                row_data["Other_Changes"].append(f"'{field_name}' field added")

    # Clean up the 'Other_Changes' list into a readable string
    row_data["Other_Changes"] = " | ".join(row_data["Other_Changes"]) if row_data["Other_Changes"] else "N/A"
    
    # Add the completed row for this file to our list
    wide_format_rows.append(row_data)


# Create and display the final DataFrame
df_wide = pd.DataFrame(wide_format_rows)

# Define the desired column order
final_columns = ["Filename", "Price", "Size", "UnitOfMeasure", "Brand", "Description", "Other_Changes"]

# Reorder the DataFrame columns. This is a safer way that avoids KeyErrors.
df_wide = df_wide.reindex(columns=final_columns)

# Rename 'UnitOfMeasure' to 'Unit' as requested
df_wide.rename(columns={"UnitOfMeasure": "Unit"}, inplace=True)
df_wide

,Filename,Price,Size,Unit,Brand,Description,Other_Changes
0,280029014909_Aldi_1.45.jpg,FAIL,FAIL,FAIL,FAIL,"[{'ground_truth': 'Organic Zucchini', 'predict...",1 product(s) removed
1,595544334785_Aldi_1.99.jpg,PASS,PASS,PASS,PASS,"[{'ground_truth': 'Garlic', 'prediction': 'Gar...",'UPC' field added
2,715756200115_Aldi_3.89.jpg,FAIL,FAIL,FAIL,FAIL,"[{'ground_truth': 'Organic Strawberries', 'pre...",1 product(s) added
3,12804_Aldi_0.89.jpg,PASS,PASS,PASS,PASS,"[{'ground_truth': 'N/A', 'prediction': 'Papaya...",N/A
4,209815404397_Aldi_0.15.jpg,PASS,FAIL,PASS,FAIL,[{'ground_truth': 'Fresh Family Pack Chicken T...,'UPC' field added
5,253097005421_Aldi_1.09.jpg,PASS,FAIL,PASS,FAIL,[{'ground_truth': 'Fresh Family Pack Chicken D...,'UPC' field added
6,253864508889_Aldi_7.59.jpg,PASS,PASS,PASS,FAIL,"[{'ground_truth': 'Extra Lean Ground Beef, 96%...",'UPC' field added
7,28400516464_Aldi_4.79.jpg,PASS,FAIL,FAIL,PASS,[{'ground_truth': 'Nacho Cheese Flavored Torti...,N/A
8,4050_Aldi_2.78.jpg,PASS,PASS,PASS,PASS,"[{'ground_truth': 'Limes', 'prediction': 'Lime...",N/A
9,4099100110005_Aldi_4.69.jpg,PASS,PASS,PASS,PASS,"[{'ground_truth': 'Center Cut Bacon', 'predict...",'UPC' field added


### Product Description Accuracy

Measures Semantic Similarity using Vertex GenAI Evaluation SDK Library. Example: "Canteloupe" vs "Canteloupe Melons"

First, a similarity score is calculated for every possible pairing between a ground truth product and a predicted product to create a master list of all potential matches.

These potential pairs are then sorted from highest score to lowest, and the best available high-scoring matches are locked in sequentially, ensuring no product is used more than once.

Finally, all remaining "unmatched" products from both lists are collected and then paired up with each other. If the number of leftovers is uneven, only the final, excess items are paired with an N/A placeholder.

In [29]:
from vertexai.evaluation import EvalTask, PointwiseMetric, PointwiseMetricPromptTemplate
import pandas as pd
from thefuzz import fuzz

In [71]:
#### EVALUATION DATASET

data_for_df = []
SIMILARITY_THRESHOLD = 70 # A score from 0-100.

# Iterate through each file in the ground truth dictionary.
for filename, gt_data in gt_dict.items():
    if filename not in pred_dict:
        # Handle cases where the entire file is missing
        for gt_product in gt_data.get('products', []):
            data_for_df.append({
                "Filename": filename,
                "response": f"{gt_product.get('Description', 'N/A')}\n\nN/A (File not found in prediction)"
            })
        continue

    gt_products = gt_data.get('products', [])
    pred_products = pred_dict[filename].get('products', [])
    
    # Create a score matrix of all possible pairings
    score_matrix = []
    for i, gt_product in enumerate(gt_products):
        for j, pred_product in enumerate(pred_products):
            gt_desc = gt_product.get('Description', '')
            pred_desc = pred_product.get('Description', '')
            
            score = fuzz.token_set_ratio(gt_desc, pred_desc) if (gt_desc or pred_desc) else 0
            score_matrix.append({'score': score, 'gt_idx': i, 'pred_idx': j})

    # Sort pairings from best score to worst
    score_matrix.sort(key=lambda x: x['score'], reverse=True)

    matched_gt_indices = set()
    matched_pred_indices = set()

    # Iterate through the sorted pairings and lock in the best ones
    for pairing in score_matrix:
        if pairing['score'] >= SIMILARITY_THRESHOLD:
            gt_idx, pred_idx = pairing['gt_idx'], pairing['pred_idx']

            if gt_idx in matched_gt_indices or pred_idx in matched_pred_indices:
                continue
            
            gt_desc = gt_products[gt_idx].get('Description', 'N/A')
            pred_desc = pred_products[pred_idx].get('Description', 'N/A')
            
            data_for_df.append({"Filename": filename, "response": f"{gt_desc}\n\n{pred_desc}"})

            matched_gt_indices.add(gt_idx)
            matched_pred_indices.add(pred_idx)

    # Collect and leftovers pairs
    
    unmatched_gt = [p for i, p in enumerate(gt_products) if i not in matched_gt_indices]
    unmatched_pred = [p for i, p in enumerate(pred_products) if i not in matched_pred_indices]

    # Iterate through the leftovers and pair them up
    max_leftovers = max(len(unmatched_gt), len(unmatched_pred))
    for i in range(max_leftovers):
        gt_desc = unmatched_gt[i].get('Description', 'N/A') if i < len(unmatched_gt) else "N/A (Product not in ground truth)"
        pred_desc = unmatched_pred[i].get('Description', 'N/A') if i < len(unmatched_pred) else "N/A (Product not in prediction)"
        
        data_for_df.append({"Filename": filename, "response": f"{gt_desc}\n\n{pred_desc}"})


# --- 4. FINAL DATAFRAME CREATION ---
df = pd.DataFrame(data_for_df)
eval_df = df.sort_values(by="Filename").reset_index(drop=True)


In [77]:
########### EVAL

custom_text_quality = PointwiseMetric(
    metric="custom_text_quality",
    metric_prompt_template=PointwiseMetricPromptTemplate(
        criteria={
            "Product Type Similairty": (
                "String comparison that matches product logically"
            ),
        },
        rating_rubric={
            "0": "The response includes 'N/A' AND/OR consists of pairs of strings that DO NOT connect (e.g. 'Cucumbers\n\nN/A (Product not in prediction)') ",
            "1": "The response consists of pairs of strings that DO connect (e.g. 'Cantaloupe Melons\n\nLoose Cantaloupe Melons') ",        
        },
    ),
)

eval_task = EvalTask(
    dataset=eval_df,
    metrics=[custom_text_quality],
)

pointwise_result = eval_task.evaluate()

The `input_variables` parameter is empty. Only the `response` column is used for computing this model-based metric.
Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.


100%|██████████| 20/20 [00:09<00:00,  2.13it/s]

All 20 metric requests are successfully computed.
Evaluation Took:9.398321671003941 seconds


In [80]:
df_eval_results = pointwise_result.metrics_table

# Assign a sequential product number within each Filename group
df_eval_results['product_num'] = df_eval_results.groupby('Filename').cumcount() + 1

# Create the desired column names (e.g., 'product_1', 'product_2').
df_eval_results['product_col_name'] = 'product_' + df_eval_results['product_num'].astype(str)

# Pivot the DataFrame.
df_eval_results = df_eval_results.pivot_table(
  index='Filename',
  columns='product_col_name',
  values='custom_text_quality/score'
)

# Clean up the resulting DataFrame
df_eval_results = df_eval_results.reset_index()

# Remove the name of the columns index for a cleaner appearance.
df_eval_results.columns.name = None

# We get the list of product columns to apply this change to.
product_cols = [col for col in df_eval_results.columns if col.startswith('product_')]
# Fill NaN with a placeholder (like -1) before converting to int, then handle it.
# NaN appears for files that have fewer products than the max.
df_eval_results[product_cols] = df_eval_results[product_cols].fillna(-1).astype(int)
# Now replace the placeholder with a more intuitive "N/A" to show no product existed.
df_eval_results.replace(-1, 'N/A', inplace=True)


# Define the mapping of values to replace.
replacement_map = {1: 'PASS', 0: 'FAIL'}

# Apply the replacement to only the product columns.
df_eval_results[product_cols] = df_eval_results[product_cols].replace(replacement_map)

# Display the final wide DataFrame
df_eval_results

,Filename,product_1,product_2
0,12804_Aldi_0.89.jpg,FAIL,N/A
1,209815404397_Aldi_0.15.jpg,PASS,N/A
2,253097005421_Aldi_1.09.jpg,PASS,N/A
3,253864508889_Aldi_7.59.jpg,PASS,N/A
4,280029014909_Aldi_1.45.jpg,PASS,FAIL
5,28400516464_Aldi_4.79.jpg,PASS,N/A
6,4050_Aldi_2.78.jpg,PASS,PASS
7,4099100110005_Aldi_4.69.jpg,PASS,N/A
8,4099200138800_Aldi_3.79.jpg,PASS,N/A
9,55124142358_Aldi_4.85.jpg,PASS,PASS


## Combine Accuracy Tables 

In [81]:
final_df = pd.merge(df_wide, df_eval_results, on='Filename', how='inner')

# Get all 'product_' columns dynamically and sort them
product_cols = sorted([col for col in final_df.columns if col.startswith('product_')])

# Define the final column order
final_column_order = [
    'Filename',
    'Price',
    'Size',
    'Unit',
    'Brand'
] + product_cols + [
    'Description',
    'Other_Changes'
]

# Apply the new order.
final_df = final_df[final_column_order]
final_df

,Filename,Price,Size,Unit,Brand,product_1,product_2,Description,Other_Changes
0,280029014909_Aldi_1.45.jpg,FAIL,FAIL,FAIL,FAIL,PASS,FAIL,"[{'ground_truth': 'Organic Zucchini', 'predict...",1 product(s) removed
1,595544334785_Aldi_1.99.jpg,PASS,PASS,PASS,PASS,PASS,N/A,"[{'ground_truth': 'Garlic', 'prediction': 'Gar...",'UPC' field added
2,715756200115_Aldi_3.89.jpg,FAIL,FAIL,FAIL,FAIL,PASS,FAIL,"[{'ground_truth': 'Organic Strawberries', 'pre...",1 product(s) added
3,12804_Aldi_0.89.jpg,PASS,PASS,PASS,PASS,FAIL,N/A,"[{'ground_truth': 'N/A', 'prediction': 'Papaya...",N/A
4,209815404397_Aldi_0.15.jpg,PASS,FAIL,PASS,FAIL,PASS,N/A,[{'ground_truth': 'Fresh Family Pack Chicken T...,'UPC' field added
5,253097005421_Aldi_1.09.jpg,PASS,FAIL,PASS,FAIL,PASS,N/A,[{'ground_truth': 'Fresh Family Pack Chicken D...,'UPC' field added
6,253864508889_Aldi_7.59.jpg,PASS,PASS,PASS,FAIL,PASS,N/A,"[{'ground_truth': 'Extra Lean Ground Beef, 96%...",'UPC' field added
7,28400516464_Aldi_4.79.jpg,PASS,FAIL,FAIL,PASS,PASS,N/A,[{'ground_truth': 'Nacho Cheese Flavored Torti...,N/A
8,4050_Aldi_2.78.jpg,PASS,PASS,PASS,PASS,PASS,PASS,"[{'ground_truth': 'Limes', 'prediction': 'Lime...",N/A
9,4099100110005_Aldi_4.69.jpg,PASS,PASS,PASS,PASS,PASS,N/A,"[{'ground_truth': 'Center Cut Bacon', 'predict...",'UPC' field added


In [82]:
# Export to JSON & CSV
final_df.to_csv('analysis_gemini2_5_pro.csv', index=False)

## Statistical Signal

This section presents the results of one-sample t-tests conducted to assess whether the true average performance for in-store images reliably exceeds the 90% benchmark. For each model tested, we confirm statistical significance (at a 95% confidence interval), indicating that performance is reliably above 90%. 

This analysis was conducted independently for various Gemini models (e.g. Flash and Pro)


In [ ]:
# To-do